In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from pathlib import Path

In [2]:
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
with Path("input.txt").open("r", encoding='utf-8') as f:
    text = f.read()

vocab = sorted(list(set(text)))
stoi = { ch: i for i, ch in enumerate(vocab) }
itos = { i: ch for i, ch in enumerate(vocab) }
encode = lambda x: [stoi[s] for s in x]
decode = lambda x: [itos[s] for s in x]

In [4]:
print("".join(decode(encode("hii there"))))

hii there


In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [6]:
vocab_size = len(vocab)
batch_size = 4
block_size = 8
n_embd = 16

max_iters = 3000
eval_iters = 200

In [7]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

get_batch('train')

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [8]:
class BLM(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        tok_emb = self.token_embedding_table(idx)
        logits = self.lm_head(tok_emb)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) 
        return logits, loss
        
    def generate(self, idx, max_new_tokens=100):
        for i in range(max_new_tokens):
            logits, _ = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat([idx, idx_next], dim=1)
        return idx

In [9]:
xb, yb = get_batch('train')

In [10]:
model = BLM()
m = model.to(device)

In [11]:
logits, loss = m(xb, yb)

In [12]:
in_ = torch.zeros((1, 1), dtype=torch.long)
out = m.generate(in_, max_new_tokens=100)
print("".join(decode(out[0].tolist())))


iWwXx!p''EetarY
;lSgQXLigIPQsWBzd.'bD:KroIAA?JyuoaOc.uCnNT,sJm:olnkQXxfILy,Afy$PWibDHS
'YZt&grKiqWaZ


In [13]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters) 
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [14]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)
for iter_ in range(max_iters):
    if iter_ % eval_iters == 0:
        losses = estimate_loss()
        print(f"step {iter_} train_loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step 0 train_loss 4.2814, val loss 4.2810
step 200 train_loss 3.7105, val loss 3.7123
step 400 train_loss 3.3813, val loss 3.3753
step 600 train_loss 3.1685, val loss 3.1825
step 800 train_loss 3.0090, val loss 2.9845
step 1000 train_loss 2.9172, val loss 2.8616
step 1200 train_loss 2.8120, val loss 2.8034
step 1400 train_loss 2.7681, val loss 2.7794
step 1600 train_loss 2.7062, val loss 2.7132
step 1800 train_loss 2.6989, val loss 2.6880
step 2000 train_loss 2.6645, val loss 2.6871
step 2200 train_loss 2.6574, val loss 2.6577
step 2400 train_loss 2.6292, val loss 2.6300
step 2600 train_loss 2.6162, val loss 2.6734
step 2800 train_loss 2.5837, val loss 2.6314
2.437225818634033
